<a href="https://colab.research.google.com/github/mkamwana/deep-learning-challenge/blob/main/Starter_Code/AlphabetSoupCharity_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preprocessing

In [1]:
pip install tensorflow

In [3]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df=application_df.drop(columns=["EIN", "NAME", "ORGANIZATION", "STATUS", "APPLICATION_TYPE"])
application_df

,AFFILIATION,CLASSIFICATION,USE_CASE,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,Independent,C1000,ProductDev,0,N,5000,1
1,Independent,C2000,Preservation,1-9999,N,108590,1
2,CompanySponsored,C3000,ProductDev,0,N,5000,0
3,CompanySponsored,C2000,Preservation,10000-24999,N,6692,1
4,Independent,C1000,Heathcare,100000-499999,N,142590,1
...,...,...,...,...,...,...,...
34294,Independent,C1000,ProductDev,0,N,5000,0
34295,CompanySponsored,C3000,ProductDev,0,N,5000,0
34296,CompanySponsored,C2000,Preservation,0,N,5000,0
34297,Independent,C3000,ProductDev,0,N,5000,1


In [5]:
# Determine the number of unique values in each column.
application_df.nunique()

AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [97]:
# Look at APPLICATION_TYPE value counts for binning
#application_df.APPLICATION_TYPE.value_counts()

In [6]:
# Look at CLASSIFICATION value counts for binning
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df["CLASSIFICATION"].value_counts().loc[lambda x: x>1]


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace =application_df["CLASSIFICATION"].value_counts().loc[lambda x: x<10].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Other       98
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
app_dummies_df=pd.get_dummies(application_df)
app_dummies_df

,ASK_AMT,IS_SUCCESSFUL,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,CLASSIFICATION_C1000,CLASSIFICATION_C1200,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,5000,1,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
1,108590,1,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,5000,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,6692,1,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,142590,1,0,0,1,0,0,0,1,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,5000,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
34295,5000,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,5000,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,5000,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [10]:
# Split our preprocessed data into our features and target arrays
X=app_dummies_df.drop(columns=["IS_SUCCESSFUL"]).values
y=app_dummies_df["IS_SUCCESSFUL"].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test= train_test_split(X,y, random_state= 1)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim= input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=10, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                4320      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 10)                310       
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 7,071
Trainable params: 7,071
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.6078 - accuracy: 0.6897
Epoch 2/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5956 - accuracy: 0.6984
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5937 - accuracy: 0.6986
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5926 - accuracy: 0.6999
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5916 - accuracy: 0.7006
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5918 - accuracy: 0.7013
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5907 - accuracy: 0.7009
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5905 - accuracy: 0.6999
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5904 - accuracy: 0.7014
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5896 - accura

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.6084 - accuracy: 0.7031 - 501ms/epoch - 2ms/step
Loss: 0.6084151268005371, Accuracy: 0.7030903697013855


In [16]:
# Export our model to HDF5 file
nn.save('Resources/AlphabetSoupCharity.h5')